In [3]:
from ib_insync import * 
import pandas as pd
import numpy as np
import datetime as dt
from IPython.display import display, clear_output
util.startLoop() 

In [4]:
ib = IB()
ib.connect(clientId=9)

<IB connected to 127.0.0.1:7497 clientId=9>

In [3]:
contract = Forex('EURUSD')
ib.qualifyContracts(contract)
cfd = CFD("EUR", currency = "USD")
ib.qualifyContracts(cfd)
conID = cfd.conId

In [4]:
trading_units = 1000
end_time = dt.time(23, 30, 0)

In [5]:
ema_s = 5
ema_l = 8

In [6]:
def onBarUpdate(bars, hasNewBar):  
    global df, last_bar
    if bars[-1].date > last_bar: # if bar completed / new bar
        last_bar = bars[-1].date
    
        df = pd.DataFrame(bars)[["date", "open", "high", "low", "close"]].iloc[:-1] 
        df.set_index("date", inplace = True)
        
        df['EMA_S'] = df.close.ewm(span = ema_s, min_periods = ema_s).mean()
        df['EMA_L'] = df.close.ewm(span = ema_l, min_periods = ema_l).mean()
        df.dropna(inplace = True)
        df['MACD'] = df['EMA_S'] - df['EMA_L']
        df['MACD_signal'] = df['MACD'].ewm(span = 3, min_periods = 3).mean()
        df['position'] = np.where(df.MACD_signal - df.MACD > 0, 1, -1)
        
        target = df['position'][-1] * trading_units
        trade_execute(target)
        
        clear_output(wait=True)
        display(df)
     
    else:
        clear_output(wait=True)
        trade_reporting()

In [7]:
def trade_execute(target): 
    try:
        current_pos = [pos.position for pos in ib.positions() if pos.contract.conId == conId][0]
    except:
        current_pos = 0
        
    trades = target - current_pos
    print(trades,target,current_pos)
    if trades < 0:
        side = 'SELL'
        order = MarketOrder(side, abs(trades))
        trade = ib.placeOrder(cfd, order)
        
        
    elif trades > 0:
        side = 'BUY'
        order = MarketOrder(side, abs(trades))
        trade = ib.placeOrder(cfd, order)
    else: 
        pass

In [8]:
def trade_reporting():
    global report
    
    fill_df = util.df([fs.execution for fs in ib.fills()])[["execId", "time", "side", "cumQty", "avgPrice"]].set_index("execId")
    profit_df = util.df([fs.commissionReport for fs in ib.fills()])[["execId", "realizedPNL"]].set_index("execId")
    report = pd.concat([fill_df, profit_df], axis = 1).set_index("time").loc[session_start:]
    report = report.groupby("time").agg({"side":"first", "cumQty":"max", "avgPrice":"mean", "realizedPNL":"sum"})
    report["cumPNL"] = report.realizedPNL.cumsum()
    
    clear_output(wait=True)
    display(report)
    
    

In [ ]:
session_start = pd.to_datetime(dt.datetime.utcnow()).tz_localize("utc")
bars = ib.reqHistoricalData(
        contract,
        endDateTime='',
        durationStr='1 D',
        barSizeSetting='1 min',
        whatToShow='MIDPOINT',
        useRTH=True,
        formatDate=2,
        keepUpToDate=True)
last_bar = bars[-1].date
bars.updateEvent += onBarUpdate

# stop trading session
while True:
    ib.sleep(5) # check every 5 seconds
    if dt.datetime.utcnow().time() >= end_time: # if stop conditions has been met
        trade_execute(target = 0) # close open position 
        ib.cancelHistoricalData(bars) # stop stream
        ib.sleep(10)
        try:
            trade_reporting() # final reporting
        except:
            pass
        print("Session Stopped.")
        ib.disconnect()
        break
    else:
        pass

,side,cumQty,avgPrice,realizedPNL,cumPNL
time,,,,,
2023-09-06 15:37:00+00:00,SLD,1000.0,1.07145,0.00000,0.00000
2023-09-06 15:38:00+00:00,SLD,1000.0,1.07155,0.00000,0.00000
2023-09-06 15:39:00+00:00,SLD,1000.0,1.07145,0.00000,0.00000
2023-09-06 15:40:00+00:00,BOT,1000.0,1.07140,-3.91667,-3.91667
2023-09-06 15:41:01+00:00,BOT,1000.0,1.07145,-3.96667,-7.88334
2023-09-06 15:42:00+00:00,BOT,1000.0,1.07140,-3.91666,-11.80000


In [5]:
ib.disconnect()


In [6]:
ib.isConnected

<bound method IB.isConnected of <IB not connected>>